In [11]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.preprocessing import Normalizer
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
#sl:satisfaction_level-False:MinMaxScaler,True:StandardScaler
#le:last_evaluation-False:MinMaxScaler,True:StandardScaler
#npr:number_project-False;MinMaxScaler,True:StandardScaler
#amh:average_monthly-hours-False;MinMaxScaler,True:StandardScaler
#tsc:time_spend_company-False;MinMaxScaler,True:StandardScaler
#wa:work_accident-False;MinMaxScaler,True:StandardScaler
#pl5:promotion_last_5years-False;MinMaxScaler,True:StandardScaler
#dp:department-False:LabelEncoding,True:OneHotEncoding
#slr；salary-False:LabelEncoding,True:OneHotEncoding
#lower_d指定是否需要降维，如需要默认降到一维
def hr_preprocessing(sl=False,le=False,npr=False,amh=False,tsc=False,wa=False,pl5=False,dp=False,slr=False,lower_d=False,ld_n=1):
    df=pd.read_csv('./data/HR.csv')
   
    #1.清洗数据(去除异常值、抽样)
    df=df.dropna(subset=['satisfaction_level','last_evaluation'])
    df=df[df['satisfaction_level']<=1][df['salary']!='nme']
     #2。得到标注
    label = df['left']
    df=df.drop('left',axis=1)#axis=1指定以列删除
    #3.特征选择
    #4.特征处理  
    scaler_lst=[sl,le,npr,amh,tsc,wa,pl5]
    column_lst=['satisfaction_level','last_evaluation','number_project','average_monthly_hours','time_spend_company','Work_accident','promotion_last_5years']
    for i in range(len(scaler_lst)):
        if not scaler_lst[i]:
            df[column_lst[i]]=\
                MinMaxScaler().fit_transform(df[column_lst[i]].values.reshape(-1,1)).reshape(1,-1)[0]#二维格式
        else:
            df[column_lst[i]]=\
                StandardScaler().fit_transform(df[column_lst[i]].values.reshape(-1,1)).reshape(1,-1)[0]
      
    scaler_lst=[dp,slr]
    column_lst=['department','salary']
    for i in range(len(scaler_lst)):
        if not scaler_lst[i]:
            if column_lst[i]=='salary':
                df[column_lst[i]]=[map_salary(s) for s in df['salary'].values]
            else:
                df[column_lst[i]]=LabelEncoder().fit_transform(df[column_lst[i]])
            df[column_lst[i]]=MinMaxScaler().fit_transform(df[column_lst[i]].values.reshape(-1,1)).reshape(1,-1)[0]
        else:
            df=pd.get_dummies(df,columns=[column_lst[i]])
    if lower_d:
        return PCA(n_components=ld_n).fit_transform(df,values).label#PVA降维，可不使用标注
    return df,label
#使salary返回对应的等级而不是012
d=dict([('low',0),('medium',1),('high',2)])
def map_salary(s):
    return d.get(s,0)   
def main():
    features,label=hr_preprocessing()
    #print(hr_preprocessing())
    #print(hr_processing(sl=True,le=True,npr=True,dp=True,lower_d=True,ld_n=3))
if __name__=='__main__':
    main()

(       satisfaction_level  last_evaluation  number_project  \
0                0.318681         0.265625             0.0   
1                0.780220         0.781250             0.6   
2                0.021978         0.812500             1.0   
3                0.692308         0.796875             0.6   
4                0.307692         0.250000             0.0   
...                   ...              ...             ...   
14994            0.340659         0.328125             0.0   
14995            0.307692         0.187500             0.0   
14996            0.307692         0.265625             0.0   
14997            0.021978         0.937500             0.8   
14998            0.307692         0.250000             0.0   

       average_monthly_hours  time_spend_company  Work_accident  \
0                   0.285047               0.125            0.0   
1                   0.775701               0.500            0.0   
2                   0.822430               0.250     